In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import os
import re
path = "C:\\Users\\Elite\\Downloads\\chromedriver_win32\\chromedriver"


#### 카테고리 제품 최대 페이지 수 ####

#### 링크 가져오기 ####
def get_link(category):

    #### 카테고리 제품 최대 페이지 수 ####
    driver = webdriver.Chrome(path)
    driver.get("https://www.iherb.com/search?kw="+category+"&sr=2&ratings=5%2C4%2C3%2C2%2C1&p=1")
    driver.implicitly_wait(10)
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    page = soup.select("a.pagination-link")
    num = page[-1].text
    parse_num = re.sub('\n','',num)
    parse_num = min(int(parse_num), 6)
    
    product_link = []
    
    for i in range(1,parse_num+1):
        driver.get("https://www.iherb.com/search?kw="+category+"&sr=2&ratings=5%2C4%2C3%2C2%2C1&p="+str(i))
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        http = soup.select("div.product-inner.product-inner-wide > a.product-image")
        for i in http:
            product_link.append(i.get('content'))
        
    return product_link

    
def get_total_link(category):
    #링크 만드는데 필요한 상품명 모음
    name = []
    a = get_link(category)
    for i in range(len(a)):
        name.append(a[i].split('/')[4])
    
    # 상품코드숫자만 모음
    code = []
    for n in range(len(a)):
        code.append(a[n].split('/')[5])

    # 상품명과 상품코드숫자를 합쳐서 링크를 만듬 (pr은 제품클림, r로 바꾸면 리뷰창으로 바로 들어감)
    total_link = [] # 링크 모음
    for k in range(len(a)):
        total_link.append('https://www.iherb.com/r/' + name[k] +"/"+ code[k])
    return total_link

    driver.close()
    
    
#### 실제 제품명 ####
def get_name(category):
    driver = webdriver.Chrome(path)
    driver.get("https://www.iherb.com/search?kw="+category+"&sr=2&ratings=5%2C4%2C3%2C2%2C1&p=1")
    driver.implicitly_wait(10)
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    page = soup.select("a.pagination-link")
    num = page[-1].text
    parse_num = re.sub('\n','',num)
    parse_num = min(int(parse_num), 6)
    
    name = []
    
    for i in range(1,parse_num+1):
        driver.get("https://www.iherb.com/search?kw="+category+"&sr=2&ratings=5%2C4%2C3%2C2%2C1&p="+str(i))
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        http = soup.select("div.product-inner.product-inner-wide > a.product-image")
    
        for i in http:
            name.append(i.get('title'))
    return name

#### 데이터프레임을 엑셀로 변환 ####

def to_excel(category):
    writer = pd.ExcelWriter('iherb_two'+category+'.xlsx')
    total_df.to_excel(writer,'sheet1')
    writer.save()
    

In [2]:
category_all = [ 'Amino Acids']

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from pandas import Series, DataFrame
import os
path = "C:\\Users\\Elite\\Downloads\\chromedriver_win32\\chromedriver"

for category in category_all:
    print(category)
    total_url = get_total_link(category) 
    name = get_name(category) 
    
    comments_all = []
    for i in range(len(total_url)):
        driver = webdriver.Chrome(path)
        driver.implicitly_wait(10)
        print(total_url[i])
        driver.get(total_url[i])
        reviews_list = []
        pages = driver.find_element_by_css_selector('div.paging-counts').text
        pages = pages[8:-6]
        print(pages)
        if pages != '' and int(pages) >= 10:
            pages = int(pages)//10
        else:
            pages = 1
    
        pages = min(pages, 10)
        #driver.close()
    
        for n in range(pages):
            html = driver.page_source
            soup = BeautifulSoup(html, "lxml")
            reviews = soup.select("div.review-core-section > div.review-text")
        
            reviews_list.extend([v.get_text() for v in reviews])
        
            button = driver.find_element_by_css_selector("div.paging > button.arrow-button > i.icon-recentlyviewedarrowright")
            driver.execute_script("arguments[0].click();",button)
            time.sleep(8)
            #driver.close()
        comments_all.append(reviews_list)
    
    review_df = pd.DataFrame(comments_all)
    name_df = pd.DataFrame(name, columns = ['name'])
    total_df = name_df.join(review_df)
    
    to_excel(category)

Amino Acids
https://www.iherb.com/r/Now-Foods-L-Cysteine-500-mg-100-Tablets/538
251
https://www.iherb.com/r/Now-Foods-L-Arginine-1-000-mg-120-Tablets/411
196
https://www.iherb.com/r/California-Gold-Nutrition-L-Glutamine-Powder-AjiPure-Gluten-Free-16-oz-454-g/71027
92
https://www.iherb.com/r/Coconut-Secret-The-Original-Coconut-Aminos-Soy-Free-Seasoning-Sauce-8-fl-oz-237-ml/24098
477
https://www.iherb.com/r/MRM-Acetyl-L-Carnitine-500-mg-60-Vegan-Capsules/41351
196
https://www.iherb.com/r/Now-Foods-L-Tyrosine-500-mg-120-Capsules/836
330
https://www.iherb.com/r/Now-Foods-Glycine-1-000-mg-100-Veg-Capsules/18106
31
https://www.iherb.com/r/Doctor-s-Best-NAC-Detox-Regulators-60-Veggie-Caps/41653
182
https://www.iherb.com/r/Now-Foods-Taurine-Double-Strength-1-000-mg-100-Veg-Capsules/15787
207
https://www.iherb.com/r/EVLution-Nutrition-Lean-Mode-Stimulant-Free-Fat-Burner-Supplement-150-Capsules/72004
115
https://www.iherb.com/r/Now-Foods-L-Lysine-1-000-mg-100-Tablets/10523
164
https://www.iherb.

51
https://www.iherb.com/r/Scivation-Xtend-HydraSport-Hydration-BCAAs-Blue-Raspberry-12-2-oz-345-g/85399

https://www.iherb.com/r/California-Gold-Nutrition-L-Glutamine-AjiPure-Gluten-Free-1000-mg-60-Veggie-Caps/72318
13
https://www.iherb.com/r/ALLMAX-Nutrition-L-Carnitine-Tartrate-Vitamin-B5-735-mg-120-Capsules/67665
19
https://www.iherb.com/r/Now-Foods-True-Calm-90-Veggie-Caps/841
200
https://www.iherb.com/r/Body-Fortress-100-Pure-Glutamine-Powder-10-6-oz-300-g/61412
35
https://www.iherb.com/r/Emergen-C-Electro-Mix-Electrolyte-Replacement-Drink-Mix-Natural-Lemon-Lime-30-Packets-0-14-oz-4-g-Each/17246
75
https://www.iherb.com/r/Country-Life-Glycine-500-mg-100-Tablets/1672
15
https://www.iherb.com/r/Now-Foods-Sports-Beta-Alanine-Endurance-750-mg-120-Veg-Capsules/13949
32
https://www.iherb.com/r/Now-Foods-L-Phenylalanine-500-mg-120-Veggie-Caps/736
57
https://www.iherb.com/r/Jarrow-Formulas-Theanine-200-200-mg-60-Veggie-Caps/13640
99
https://www.iherb.com/r/Source-Naturals-L-Lysine-1-000-

In [4]:
review_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,I have used this product to reduce inflammatio...,I didn't get those today. please delivery ASAP!!!,"Smells bad but seems to be working, getting le...",I had a flu after I received the package and t...,Cured cold in no time,This product is the best Propolis extract that...,GET THIS. Eat it when you feel the onset of a ...,The result not different,So far so great! This is my first times using ...,Good,...,good,I prefer black propolis liquid alcoholic extr...,之前有一點鼻炎跟塞，剛好食了幾天發現通了，也不肯定是不是這蜂膠丸起作用，就繼續食下，胶囊味有...,Not feels difference!,I ordered this brand of bee Propolis second ...,"only one product sent was powder, which it so ...",I read that propolis can help prevent getting ...,Good for immune system,Before I would have an outbreak every month. I...,그럭저럭
1,good for immunity,Good quality,good!good!good!good!good!good!,Got pharyngitis even antibiotics cant heal. Th...,My mother loves this product.,GOOD,the function is good\nI cannot feel some diffe...,I used to take this regularly and then switche...,"I like it. Can`t say is it working or not, bec...",EXCELLENT,...,good,good,I bought this many times for my parents.\nThey...,"i used other products of propolis, liquid type...",very good,reasonable price \nsome point is that capsule ...,Capacity is too small.\n,very good,good,great works
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"Caught a cold, then coughing for few months, t...",Effective. Pricy but god.,Tried & Works Good For Me,👍👍,I do believe propolis improves your health and...,Good taste is better than other brands of prop...,I feel that it boost immunity in very strong m...,Good and will order again,It is good for health,Improve my defense,...,None,None,None,None,None,None,None,None,None,None
4,100% the best products and the best price.It's...,I do like the thought that I am taking bee pro...,No effective in allergic cold n cough,Very good product!!!,im still waiting for my order... almost SIX MO...,This is the first time I'm trying out Propolis...,GOOD QUALITY FOR PRICE,This product really helps me fight colds and f...,Caps with dried propolis. I take propolis two ...,Very good product,...,Easy to take. No gastric side-effects that I h...,I had a cough for several weeks. Read the revi...,"taste is very unpleasant, but works good.",It effect good for my daughter.,The product is still in the experience of taki...,even I given to my son it worked well.,I like this product. I use it for sinus infect...,This helps my gums which used to bleed when I ...,Not sure about the effect of this product. I h...,Propolis is the best natural immune enhancer e...
5,I have used this product to reduce inflammatio...,I didn't get those today. please delivery ASAP!!!,"Smells bad but seems to be working, getting le...",I had a flu after I received the package and t...,Cured cold in no time,This product is the best Propolis extract that...,GET THIS. Eat it when you feel the onset of a ...,The result not different,So far so great! This is my first times using ...,Good,...,good,I prefer black propolis liquid alcoholic extr...,之前有一點鼻炎跟塞，剛好食了幾天發現通了，也不肯定是不是這蜂膠丸起作用，就繼續食下，胶囊味有...,Not feels difference!,I ordered this brand of bee Propolis second ...,"only one product sent was powder, which it so ...",I read that propolis can help prevent getting ...,Good for immune system,Before I would have an outbreak every month. I...,그럭저럭
6,"I'm impressed.\nThis, single-handedly, wiped o...",Burns my throat. Maybe that's a good indicator...,this item do not in my mailbox,It tastes terrible and its color stick on my hand,I reported this to iHerb and they said their s...,taste is terrible,"great product , highly reccemendet",While some might complain about the texture of...,Propolis is a Natural antibiotic and I use it ...,I've been taking this twice everyday for years...,...,None,None,None,None,None,None,None,None,None,None
7,I like this product as I 

In [4]:
total_url

['https://www.iherb.com/r/Now-Foods-L-Cysteine-500-mg-100-Tablets/538',
 'https://www.iherb.com/r/Now-Foods-L-Arginine-1-000-mg-120-Tablets/411',
 'https://www.iherb.com/r/California-Gold-Nutrition-L-Glutamine-Powder-AjiPure-Gluten-Free-16-oz-454-g/71027',
 'https://www.iherb.com/r/Coconut-Secret-The-Original-Coconut-Aminos-Soy-Free-Seasoning-Sauce-8-fl-oz-237-ml/24098',
 'https://www.iherb.com/r/MRM-Acetyl-L-Carnitine-500-mg-60-Vegan-Capsules/41351',
 'https://www.iherb.com/r/Now-Foods-L-Tyrosine-500-mg-120-Capsules/836',
 'https://www.iherb.com/r/Now-Foods-Glycine-1-000-mg-100-Veg-Capsules/18106',
 'https://www.iherb.com/r/Doctor-s-Best-NAC-Detox-Regulators-60-Veggie-Caps/41653',
 'https://www.iherb.com/r/Now-Foods-Taurine-Double-Strength-1-000-mg-100-Veg-Capsules/15787',
 'https://www.iherb.com/r/EVLution-Nutrition-Lean-Mode-Stimulant-Free-Fat-Burner-Supplement-150-Capsules/72004',
 'https://www.iherb.com/r/Now-Foods-L-Lysine-1-000-mg-100-Tablets/10523',
 'https://www.iherb.com/r/Ca

In [6]:
name_df

,name
0,"California Gold Nutrition, 프로폴리스 2X, 농축 제품, 50..."
1,"Y.S. Eco Bee Farms, 프로폴리스 1000, 500 mg, 90 식물성 캡슐"
2,"Y.S. Eco Bee Farms, 프로폴리스, 무 알콜 1 fl oz (30 ml)"
3,"Nature's Answer, 프로폴리스, 2,000 mg, 30 ml (1 fl oz)"
4,"Now Foods, 프로폴리스 1500, 300 mg, 100 야채 캡슐"
5,"California Gold Nutrition, 프로폴리스 2X, 농축 제품, 50..."
6,"Y.S. Eco Bee Farms, 프로 폴리스, 70 % 울트라 강력함, 1 fl..."
7,"NaturaNectar, 벌 프로폴리스 트리오, 60 식물성 캡슐"
8,"Y.S. Eco Bee Farms, 로열젤리, 꿀 속 화분, 프로폴리스, 인삼, 1..."
9,"Wedderspoon, 유기농 마누카 허니 드롭, 밀랍 첨가 유칼립투스, 4 oz ..."


In [5]:
k = ['Amino Acids']
for category in k:
    name = get_name(category) 

In [7]:
len(name)

24

In [56]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import os
import re
path = "C:\\Users\\Elite\\Downloads\\chromedriver_win32\\chromedriver"
driver = webdriver.Chrome(path)
driver.implicitly_wait(10)
    
    #driver.get("https://www.iherb.com/search?kw="+category+"&sr=2&p=i")
product_link = []
k_list=[]    
for i in range(1):
    driver.get("https://www.iherb.com/search?kw=Magnesium&sr=2&p=i")
    driver.implicitly_wait(10)
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    http = soup.select("a.pagination-link")
  
    #list_ = [k.get_text() for k in http]
    num = http[-1].text
    parse_num = re.sub('\n','',text)

In [29]:
list_

['\n\n2\n', '\n\n3\n', '\n\n4\n', '\n\n14\n']

In [43]:
a = http[-1]

In [41]:
list_[-1]

'\n\n14\n'

In [51]:
a.text

'\n\n14\n'

In [52]:
import re
text = a.text

In [54]:
parse = re.sub('\n','', text)

In [55]:
parse

'14'

In [57]:
parse_num

'14'

In [60]:
for i in range(int(parse_num)):
    print('a')

a
a
a
a
a
a
a
a
a
a
a
a
a
a


In [4]:
driver = webdriver.Chrome(path)
driver.get("https://www.iherb.com/search?kw="+category+"&sr=2&ratings=5%2C4%2C3%2C2%2C1&p=1")
driver.implicitly_wait(10)
html = driver.page_source
soup = BeautifulSoup(html, "lxml")
page = soup.select("a.pagination-link")
num = page[-1].text
parse_num = re.sub('\n','',num)
    
    
product_link = []
    
for i in range(1,int(parse_num)+1):
    driver.get("https://www.iherb.com/search?kw="+category+"&sr=2&ratings=5%2C4%2C3%2C2%21C1&p="+str(i))

In [ ]:
category_all = [ 'Amino Acids', 'Bee Propolis', 'Collagen','Digestive Enzymes', 'Omega 3', 'Glucosamine', 'Greens & Superfoods',
 'Lutein', 'Turmeric & Curcumin', 'Vitamin B', 'Vitamin C', 'Vitamin D', 'Calcium', 'Iron', 'Magnesium']

In [4]:
for category in category_all:
    print(category)
    total_url = get_total_link(category) 

Amino Acids
Bee Propolis


In [5]:
total_url

['https://www.iherb.com/r/California-Gold-Nutrition-Bee-Propolis-2X-Concentrated-Extract-500-mg-240-Veggie-Caps/61840',
 'https://www.iherb.com/r/Y-S-Eco-Bee-Farms-Propolis-1000-500-mg-90-Veggie-Caps/23691',
 'https://www.iherb.com/r/Y-S-Eco-Bee-Farms-Propolis-High-Strength-Alcohol-Free-1-fl-oz-30-ml/23690',
 'https://www.iherb.com/r/Nature-s-Answer-Propolis-2-000-mg-1-fl-oz-30-ml/5166',
 'https://www.iherb.com/r/Now-Foods-Propolis-1500-300-mg-100-Veg-Capsules/762',
 'https://www.iherb.com/r/California-Gold-Nutrition-Bee-Propolis-2X-Concentrated-Extract-500-mg-90-Veggie-Caps/61839',
 'https://www.iherb.com/r/Y-S-Eco-Bee-Farms-Propolis-70-Ultra-Strength-1-fl-oz-30-ml/23688',
 'https://www.iherb.com/r/NaturaNectar-Bee-Propolis-Trio-60-Vegetable-Capsules/55817',
 'https://www.iherb.com/r/Y-S-Eco-Bee-Farms-Royal-Jelly-Pollen-Propolis-Ginseng-in-Honey-1-2-lbs-552-g/23664',
 'https://www.iherb.com/r/Wedderspoon-Organic-Manuka-Honey-Drops-Lemon-With-Bee-Propolis-4-oz-120-g/69107',
 'https://w